In [2]:
import yfinance as yf
import numpy as np
import pandas as pd

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (Close, SPY)   1551 non-null   float64
 1   (High, SPY)    1551 non-null   float64
 2   (Low, SPY)     1551 non-null   float64
 3   (Open, SPY)    1551 non-null   float64
 4   (Volume, SPY)  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


In [5]:
len(tsla)

1551

In [6]:
xly.head()

Price,Close,High,Low,Open,Volume
Ticker,XLY,XLY,XLY,XLY,XLY
Date,,,,,
2019-01-02,94.046082,94.574004,91.246192,91.745834,6840800
2019-01-03,92.009827,93.555893,91.849565,93.329641,6346000
2019-01-04,95.054832,95.752446,93.065685,93.254227,7269100
2019-01-07,97.204216,97.826410,95.460179,95.507311,6263100
2019-01-08,98.278923,98.806846,96.883689,98.212933,9391000


In [7]:
def calculate_vortex(df, value, n=14):
    high = df[("High", value)]
    low = df[("Low", value)]
    close = df[("Close", value)]

    # Calculate VM+ and VM-
    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    # Calculate True Range (TR)
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Rolling sum for lookback period
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Compute VI+ and VI-
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [8]:
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

In [9]:
spy.head(20)

Price,Close,High,Low,Open,Volume,VI+,VI-
Ticker,SPY,SPY,SPY,SPY,SPY,,
Date,,,,,,,
2019-01-02,226.954697,227.889090,223.117390,223.144604,126925200,NaN,NaN
2019-01-03,221.538971,225.494214,221.049094,225.185768,144140700,NaN,NaN
2019-01-04,228.959579,229.612740,224.224173,224.605181,142628800,NaN,NaN
2019-01-07,230.764832,232.189075,228.324553,229.231719,103139100,NaN,NaN
2019-01-08,232.932953,233.422830,230.420114,232.978328,102512600,NaN,NaN
2019-01-09,234.021591,234.874329,232.406837,233.649650,95006600,NaN,NaN
2019-01-10,234.847122,235.101128,231.780897,232.470352,96823900,NaN,NaN
2019-01-11,234.937805,234.965019,233.168821,233.758473,73858100,NaN,NaN


In [10]:
import requests
import pandas as pd

url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=PNM5EHRALIOT1CKJ'

response = requests.get(url)

if response.status_code == 200:
    sentiment_data = response.json()
    sentiment_df = pd.DataFrame(sentiment_data['feed']) 
    print(sentiment_df.head())
else:
    print("API call failed:", response.status_code)


                                               title  \
0  Looking Back on Berkshire Hathaway's Outperfor...   
1                       How to Tune Out Market Noise   
2  EXCLUSIVE: Which Magnificent 7 Stock Will Perf...   
3  New Inflation Data Dismays Bulls, CoreWeave IP...   
4  Assessing Apple's Performance Against Competit...   

                                                 url   time_published  \
0  https://www.fool.com/investing/2025/03/28/look...  20250328T211200   
1  https://www.fool.com/investing/2025/03/28/how-...  20250328T194400   
2  https://www.benzinga.com/tech/25/03/44544129/e...  20250328T190544   
3  https://www.benzinga.com/markets/equities/25/0...  20250328T161558   
4  https://www.benzinga.com/insights/news/25/03/4...  20250328T150055   

               authors                                            summary  \
0  [Motley Fool Staff]  Over the past five years, Warren Buffett's ret...   
1  [Motley Fool Staff]  In this podcast, Motley Fool analyst Asit Shar

In [11]:
sentiment_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,Looking Back on Berkshire Hathaway's Outperfor...,https://www.fool.com/investing/2025/03/28/look...,20250328T211200,[Motley Fool Staff],"Over the past five years, Warren Buffett's ret...",https://g.foolcdn.com/editorial/images/812798/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.123972,Neutral,"[{'ticker': 'KTB', 'relevance_score': '0.00890..."
1,How to Tune Out Market Noise,https://www.fool.com/investing/2025/03/28/how-...,20250328T194400,[Motley Fool Staff],"In this podcast, Motley Fool analyst Asit Shar...",https://g.foolcdn.com/editorial/images/812797/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.071763,Neutral,"[{'ticker': 'DKNG', 'relevance_score': '0.0076..."
2,EXCLUSIVE: Which Magnificent 7 Stock Will Perf...,https://www.benzinga.com/tech/25/03/44544129/e...,20250328T190544,[Chris Katje],Benzinga readers pick their favorite Magnifice...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.266216,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1984..."
3,"New Inflation Data Dismays Bulls, CoreWeave IP...",https://www.benzinga.com/markets/equities/25/0...,20250328T161558,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.240809,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1725..."
4,Assessing Apple's Performance Against Competit...,https://www.benzinga.com/insights/news/25/03/4...,20250328T150055,[Benzinga Insights],Amidst the fast-paced and highly competitive b...,https://www.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.265078,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.4680..."
5,Apple's Options Frenzy: What You Need to Know ...,https://www.benzinga.com/insights/options/25/0...,20250328T134533,[Benzinga Insights],Investors with a lot of money to spend have ta...,https://www.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.185008,Somewhat-Bullish,"[{'ticker': 'EVR', 'relevance_score': '0.06568..."
6,Trading The SPY As PCE Inflation Report Takes ...,https://www.benzinga.com/markets/equities/25/0...,20250328T125805,[RIPS],Good Morning Traders! In today's Market Clubho...,https://www.benzinga.com/next-assets/images/sc...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.084859,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.0813..."
7,EU To Issue Minimal Fines To Apple And Meta To...,https://www.benzinga.com/markets/25/03/4453457...,20250328T122729,[Namrata Sen],The European Union ( EU ) is reportedly set to...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.103777,Neutral,"[{'ticker': 'META', 'relevance_score': '0.2658..."
8,3 Northern Mutual Funds for Solid Returns,https://www.zacks.com/stock/news/2436613/3-nor...,20250328T111100,[Zacks Equity Research],"Invest in Northern mutual funds like NOIEX, NM...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.272886,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1496..."
9,2 Breakout Growth Stocks You Can Buy and Hold ...,https://www.fool.com/investing/2025/03/28/2-br...,20250328T110700,[Anders Bylund],Looking for undervalued stocks with explosive ...,https://g.foolcdn.com/editorial/images/812691/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Economy - Monetary',

## volatility

In [12]:

# Flatten MultiIndex columns 
tsla.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in tsla.columns
]

# Calculate True Range
tsla["prev_close"] = tsla["Close_TSLA"].shift(1)
tsla["tr1"] = tsla["High_TSLA"] - tsla["Low_TSLA"]
tsla["tr2"] = abs(tsla["High_TSLA"] - tsla["prev_close"])
tsla["tr3"] = abs(tsla["Low_TSLA"] - tsla["prev_close"])

tsla["true_range"] = tsla[["tr1", "tr2", "tr3"]].max(axis=1)

# 10-day ATR
tsla["ATR_10"] = tsla["true_range"].rolling(window=10).mean()

# ---- STEP 4: Calculate ATR as a percentage of closing price ----
tsla["atr_pct"] = tsla["ATR_10"] / tsla["Close_TSLA"]

# allocating the capital

def position_size(row):
    if row["atr_pct"] < 0.03:  # < 3% volatility → low risk
        return 0.01  # allocate 1% of capital
    else:  # ≥ 3% volatility → high risk
        return 0.005  # allocate 0.5% of capital

tsla["position_size"] = tsla.apply(position_size, axis=1)

# ---- STEP 6: Optional - Capital allocation per trade ----
#capital = 100000 # Example: $100K total portfolio
#tsla["allocation_dollars"] = tsla["position_size"] * capital

# ---- Preview ----
print(tsla[["Close_TSLA", "ATR_10", "atr_pct", "position_size"]].tail(10))


            Close_TSLA     ATR_10   atr_pct  position_size
Date                                                      
2025-02-19  360.559998  16.703000  0.046325          0.005
2025-02-20  354.399994  16.464999  0.046459          0.005
2025-02-21  337.799988  17.021997  0.050391          0.005
2025-02-24  330.529999  16.770996  0.050740          0.005
2025-02-25  302.799988  18.879996  0.062351          0.005
2025-02-26  290.799988  18.412994  0.063318          0.005
2025-02-27  281.950012  18.257996  0.064756          0.005
2025-02-28  292.980011  18.067996  0.061670          0.005
2025-03-03  284.649994  19.281998  0.067739          0.005
2025-03-04  272.040009  20.654996  0.075926          0.005


In [13]:
import plotly.express as px
fig = px.line(tsla, x=tsla.index, y="atr_pct", title="ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [14]:
import plotly.express as px

# Filter only 2025 data
tsla_2025 = tsla[tsla.index.year == 2025]

# Plot
fig = px.line(tsla_2025, x=tsla_2025.index, y="atr_pct", title="ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [15]:
print(tsla.columns)


Index(['Close_TSLA', 'High_TSLA', 'Low_TSLA', 'Open_TSLA', 'Volume_TSLA',
       'VI+_', 'VI-_', 'prev_close', 'tr1', 'tr2', 'tr3', 'true_range',
       'ATR_10', 'atr_pct', 'position_size'],
      dtype='object')


In [16]:
# Calculate ATR percentage
tsla['atr_pct'] = tsla['ATR_10'] / tsla['Close_TSLA']

# Create Buy Signal (assuming VI_Cross_Up is defined elsewhere)
tsla['Buy_Signal'] = tsla['VI+_'] > tsla['VI-_']  # Vortex crossover
# + add any other buy conditions here...

# Create Sell Signal (basic)
tsla['Sell_Signal'] = tsla['VI-_'] > tsla['VI+_']

# Initialize position state
tsla['Position'] = 0
peak_price = 0

for i in range(1, len(tsla)):
    if tsla['Buy_Signal'].iloc[i]:
        tsla.at[tsla.index[i], 'Position'] = 1
        peak_price = tsla['Close_TSLA'].iloc[i]
    elif tsla['Position'].iloc[i - 1] == 1:
        current_price = tsla['Close_TSLA'].iloc[i]
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03:
            tsla.at[tsla.index[i], 'Sell_Signal'] = True  # trailing stop
            tsla.at[tsla.index[i], 'Position'] = 0
        else:
            tsla.at[tsla.index[i], 'Position'] = 1


In [17]:
print("Buy signals:", tsla['Buy_Signal'].sum())
print("Sell signals:", tsla['Sell_Signal'].sum())


Buy signals: 857
Sell signals: 680


In [18]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=tsla.index, y=tsla['Close_TSLA'], mode='lines', name='TSLA Price'))

# Buy markers
fig.add_trace(go.Scatter(
    x=tsla[tsla['Buy_Signal']].index,
    y=tsla[tsla['Buy_Signal']]['Close_TSLA'],
    mode='markers',
    marker=dict(symbol='triangle-up', size=10, color='green'),
    name='Buy Signal'
))

# Sell markers
fig.add_trace(go.Scatter(
    x=tsla[tsla['Sell_Signal']].index,
    y=tsla[tsla['Sell_Signal']]['Close_TSLA'],
    mode='markers',
    marker=dict(symbol='triangle-down', size=10, color='red'),
    name='Sell Signal'
))

fig.update_layout(title='TSLA Buy & Sell Signals', template='plotly_white')
fig.show()


In [19]:
print(tsla['Buy_Signal'].sum())  # Should be > 0
print(tsla['Sell_Signal'].sum())

857
680


In [24]:
tsla = tsla.dropna(subset=['Close_TSLA'])
entries = tsla['Buy_Signal'].astype(bool)
exits = tsla['Sell_Signal'].astype(bool)


In [28]:
import vectorbt as vbt

In [29]:
price = tsla['Close_TSLA']
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

print(portfolio.stats())
portfolio.plot().show()

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                              100000.0
End Value                           162759.235978
Total Return [%]                        62.759236
Benchmark Return [%]                  1215.813231
Max Gross Exposure [%]                      100.0
Total Fees Paid                      24054.581607
Max Drawdown [%]                        55.348959
Max Drawdown Duration                       730.0
Total Trades                                   80
Total Closed Trades                            80
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 32.5
Best Trade [%]                          46.283397
Worst Trade [%]                         -9.410141
Avg Winning Trade [%]                   11.344578
Avg Losing Trade [%]                    -3.847352


In [20]:
# Calculate ATR percentage
tsla['atr_pct'] = tsla['ATR_10'] / tsla['Close_TSLA']

# Vortex crossover logic
tsla['VI_Cross_Up'] = (tsla['VI+_'] > tsla['VI-_']) & (tsla['VI+_'].shift(1) <= tsla['VI-_'].shift(1))
tsla['VI_Cross_Down'] = (tsla['VI-_'] > tsla['VI+_']) & (tsla['VI-_'].shift(1) <= tsla['VI+_'].shift(1))

# Initialize signal & state columns
tsla['Buy_Signal'] = False
tsla['Sell_Signal'] = False
tsla['Position'] = 0
tsla['Entry_Type'] = None  # aggressive/conservative

# Trailing stop logic variables
in_position = False
peak_price = 0

for i in range(1, len(tsla)):
    row = tsla.iloc[i]
    idx = tsla.index[i]

    # Buy condition
    if not in_position and row['VI_Cross_Up']:
        tsla.at[idx, 'Buy_Signal'] = True
        tsla.at[idx, 'Position'] = 1
        in_position = True
        peak_price = row['Close_TSLA']

        # Entry Type: aggressive if ATR < 3%, else conservative
        if row['atr_pct'] < 0.03:
            tsla.at[idx, 'Entry_Type'] = 'aggressive'
        else:
            tsla.at[idx, 'Entry_Type'] = 'conservative'

    # While in position, check for trailing stop or VI cross down
    elif in_position:
        current_price = row['Close_TSLA']
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03 or row['VI_Cross_Down']:
            tsla.at[idx, 'Sell_Signal'] = True
            tsla.at[idx, 'Position'] = 0
            in_position = False
        else:
            tsla.at[idx, 'Position'] = 1

# Show result counts
print("Buy signals:", tsla['Buy_Signal'].sum())
print("Sell signals:", tsla['Sell_Signal'].sum())
print("Aggressive entries:", (tsla['Entry_Type'] == 'aggressive').sum())
print("Conservative entries:", (tsla['Entry_Type'] == 'conservative').sum())


Buy signals: 80
Sell signals: 80
Aggressive entries: 5
Conservative entries: 75


In [21]:
import plotly.graph_objects as go

fig = go.Figure()

# Plot TSLA closing price
fig.add_trace(go.Scatter(
    x=tsla.index, 
    y=tsla['Close_TSLA'], 
    mode='lines', 
    name='TSLA Price', 
    line=dict(color='blue')
))

# Aggressive buys
fig.add_trace(go.Scatter(
    x=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'aggressive')].index,
    y=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'aggressive')]['Close_TSLA'],
    mode='markers',
    name='Buy (Aggressive)',
    marker=dict(symbol='triangle-up', color='limegreen', size=10)
))

# Conservative buys
fig.add_trace(go.Scatter(
    x=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'conservative')].index,
    y=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'conservative')]['Close_TSLA'],
    mode='markers',
    name='Buy (Conservative)',
    marker=dict(symbol='triangle-up', color='green', size=10)
))

# Sells
fig.add_trace(go.Scatter(
    x=tsla[tsla['Sell_Signal']].index,
    y=tsla[tsla['Sell_Signal']]['Close_TSLA'],
    mode='markers',
    name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=10)
))

fig.update_layout(
    title='TSLA Buy/Sell Signals Over Time',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    height=600
)

fig.show()


In [22]:
# Calculate ATR percentage
tsla['atr_pct'] = tsla['ATR_10'] / tsla['Close_TSLA']

# Detect VI+ crossover and VI- crossover
tsla['VI_Cross_Up'] = (tsla['VI+_'] > tsla['VI-_']) & (tsla['VI+_'].shift(1) <= tsla['VI-_'].shift(1))
tsla['VI_Cross_Down'] = (tsla['VI-_'] > tsla['VI+_']) & (tsla['VI-_'].shift(1) <= tsla['VI+_'].shift(1))

# Initialize signal and tracking columns
tsla['Buy_Signal'] = False
tsla['Sell_Signal'] = False
tsla['Position'] = 0
tsla['Entry_Type'] = None  # 'aggressive' or 'conservative'

peak_price = 0  # To track high for trailing stop

# Loop over data
for i in range(1, len(tsla)):
    row = tsla.iloc[i]
    idx = tsla.index[i]

    # --- BUY SIGNAL ---
    if row['VI_Cross_Up']:
        tsla.at[idx, 'Buy_Signal'] = True
        tsla.at[idx, 'Position'] = 1
        peak_price = row['Close_TSLA']  # Reset peak on new buy

        # Entry type classification
        if row['atr_pct'] < 0.03:
            tsla.at[idx, 'Entry_Type'] = 'aggressive'
        else:
            tsla.at[idx, 'Entry_Type'] = 'conservative'

    # --- IN TRADE: Maintain or Exit ---
    elif tsla['Position'].iloc[i - 1] == 1:
        current_price = row['Close_TSLA']
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        # --- SELL SIGNAL: Either VI- cross or trailing stop ---
        if row['VI_Cross_Down'] or drawdown >= 0.03:
            tsla.at[idx, 'Sell_Signal'] = True
            tsla.at[idx, 'Position'] = 0
        else:
            tsla.at[idx, 'Position'] = 1

    # --- NO POSITION ---
    else:
        tsla.at[idx, 'Position'] = 0

# Summary stats
print("Buy signals:", tsla['Buy_Signal'].sum())
print("Sell signals:", tsla['Sell_Signal'].sum())
print("Aggressive entries:", (tsla['Entry_Type'] == 'aggressive').sum())
print("Conservative entries:", (tsla['Entry_Type'] == 'conservative').sum())


Buy signals: 80
Sell signals: 80
Aggressive entries: 5
Conservative entries: 75


In [23]:
import plotly.graph_objects as go

fig = go.Figure()

# Price line
fig.add_trace(go.Scatter(
    x=tsla.index, y=tsla['Close_TSLA'],
    mode='lines', name='TSLA Price',
    line=dict(color='royalblue')
))

# Aggressive Buy markers (ATR% < 3%)
fig.add_trace(go.Scatter(
    x=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'aggressive')].index,
    y=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'aggressive')]['Close_TSLA'],
    mode='markers',
    name='Buy (Aggressive)',
    marker=dict(symbol='triangle-up', color='limegreen', size=10)
))

# Conservative Buy markers (ATR% >= 3%)
fig.add_trace(go.Scatter(
    x=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'conservative')].index,
    y=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'conservative')]['Close_TSLA'],
    mode='markers',
    name='Buy (Conservative)',
    marker=dict(symbol='triangle-up', color='orange', size=10)
))

# Sell markers
fig.add_trace(go.Scatter(
    x=tsla[tsla['Sell_Signal']].index,
    y=tsla[tsla['Sell_Signal']]['Close_TSLA'],
    mode='markers',
    name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=10)
))

# Layout
fig.update_layout(
    title='TSLA Buy & Sell Signals (Multi-Entry Logic)',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    legend=dict(x=0.01, y=0.99),
    height=600,
    template='plotly_white'
)

fig.show()
